In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats, integrate, optimize
import pandas as pd
import seaborn as sns
import calendar
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
pd.options.display.max_rows = 400
pd.options.display.max_columns = 400

In [53]:
new = pd.read_excel("Results_Updated_2014_2015.xlsx")
demo = pd.read_excel("demo_cleaned_incomplete_07_04_2016.xlsx")

In [54]:
data = new[["Order", "Date", "Ward Name", "RNONE"]]
data.ix[data.RNONE == "RNONE", "RNONE"] = "Negative"
data.ix[data.RNONE != "Negative", "RNONE"] = "Positive"
data = data.rename(columns={"RNONE":"Type"})
data = data.rename(columns={"Order":"Order_ID"})
data = data.rename(columns={"Ward Name":"Ward_Name"})
data["Positive"] = np.nan
data = data.reset_index(drop=True)
data.ix[data.Type == "Positive", "Positive"] = 1
data.ix[data.Type == "Negative", "Positive"] = 0
data.Date = data.Date.astype("str")
data.Positive = data.Positive.astype(int)

In [55]:
posdata = new[["Order", "Date", "Ward Name", "RNONE","RBHOM", "RDFRG", "RNANA", "RCOLI", "RHART", "RCHIL", "RGIAR", "RHIS2", "RIODA", "RTRIC", "EIAG", "EIAC", "EIAH"]]
posdata = posdata.reset_index(drop=True)

In [57]:
organisms = {}
for x in posdata.values:
    l = []
    for i, y in enumerate(x):
        if (str(y) != 'nan') & (i >=3):
            l.append(y)
    organisms[x[0]] = l
#organisms 

In [58]:
a = pd.Series(list(organisms.values()))
b = pd.Series(list(organisms.keys()))
c = pd.concat([b,a], axis=1)
c = c.rename(columns = {0:"Order_ID", 1:"Organism"})
c.Organism = c.Organism.apply(lambda x: ", ".join([i[1:] if (i[0] == "R" or i[0] == "2") else i[:] for i in x]))

In [59]:
data = data.merge(c, how="outer", on="Order_ID")
data.Date = pd.to_datetime(data.Date)
data = data.merge(demo, left_on="Order_ID", right_on="Order").drop("Order", axis=1)
data = data.drop(["Doctor", "Doc_Ward", "Result", "State", "Doc_City1", "Doc_City2", "Doc_Location"], axis=1)

In [77]:
for x in data.Date.dt.year.unique():
    for i in data[data.Date.dt.year == x].Date.dt.month.unique():
        d = data[(data.Date.dt.year == x) & (data.Date.dt.month == i)].reset_index(drop=True)
        d.Date = d.Date.astype(str)
        d.to_json("data_{0}_{1}.json".format(x, calendar.month_name[i].lower()), orient="records")